In [1]:
import numpy as np
import datajoint as dj
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
import ipyvolume.pylab as p3
from collections import Counter
import time
import pandas as pd
import seaborn as sns
import random
# import tsne
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import gaussian_kde
from numpy import linalg

In [2]:
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')
nda = dj.create_virtual_module('nda2', 'microns_nda2')
dj.config['display.limit'] = 15

Connecting celiib@10.28.0.34:3306


In [3]:
#1) Found all of the axon segments

#gets the segments id's of all of the segments that are axons
#ta3.Neurite() just has the segment_id and whether it is a dendrite or axon
axon_segments = (ta3.Neurite() & "neurite_type='axon'").proj(presyn="segment_id")

In [4]:
#ta3.Segment : has segment id and previous segment id
#ta3.Soma : has the semgnet id of the soma and the EM id
#nda.Trace : EM id and the calcium trace
post_trace_segments = (ta3.Segment & nda.Trace & ta3.Soma).proj(postsyn = 'segment_id')

In [16]:
#ta3.Synapse() has all of the synapse data: id, presynaptic, post-synaptic, coordinates and bounding box coordinates
#axon_segments --> restricts so only valid axons can be the presynaptic targets
#post_trace_segments --> restricts to only post-synaptic targets that have a valid soma and calcium trace
axon_trace_synapses = ta3.Synapse() & axon_segments & post_trace_segments
 v

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),syn_bbox_x1 (EM voxels) - bounding box,syn_bbox_y1 (EM voxels) - bounding box,syn_bbox_z1 (EM voxels) - bounding box,syn_bbox_x2 (EM voxels) - bounding box,syn_bbox_y2 (EM voxels) - bounding box,syn_bbox_z2 (EM voxels) - bounding box
1,21460,54101819,36456160,10341,21973,833,10324,21946,832,10359,22007,835
1,21485,50165577,36456160,10393,21700,836,10336,21670,834,10442,21746,842
1,23369,58116980,36456160,10645,22356,820,10614,22323,820,10676,22409,822
1,23579,54037914,36456160,10794,22280,818,10763,22263,815,10828,22300,822
1,23689,54101819,36456160,10902,22185,828,10860,22135,826,10940,22225,832
1,23728,54034799,36456160,10928,22395,757,10883,22368,755,10978,22427,760
1,23805,58036847,36456160,10965,22514,751,10930,22498,750,10999,22532,752
1,23870,50165937,36456160,11074,22097,848,10977,22055,842,11174,22141,857
1,25181,58037816,36456160,10714,23684,764,10697,23663,763,10728,23706,768
1,25797,31523985,36456160,11109,23430,747,11091,23406,745,11125,23458,751


In [18]:
axon_trace_synapses = axon_trace_synapses * ta3.SynapseCompartmentLabel
#reduces the number by about 1000

In [19]:
axons_from_unique_axon_trace_pairs_table = dj.U('presyn','postsyn').aggr(axon_trace_synapses, n_syn='count(*)')
axons_from_unique_axon_trace_pairs_table 

presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,n_syn calculated attribute
108,20152808,1
9117,82251040,1
18695,82251040,1
20560,74119606,1
21736,82251040,1
27408,3851918,1
27408,23683777,1
31276,82251040,1
35065,694582,1
35324,694582,1


In [20]:
scan_idx = 3
num_postsyns = 4
von_r2_min = 0.05
von_pvalue_max = 0.05

In [21]:
is_tuned = 'von_r2>{} and von_pvalue<{}'.format(str(von_r2_min),str(von_pvalue_max))

In [22]:
#getting the functional data for any postsynaptic
#gets the directional preferences of only significanly tuned neurons and gets their postsyn id
functional_data = (nda.VonMises & is_tuned).proj('von_pref',postsyn='segment_id') & 'scan_idx={}'.format(str(scan_idx))
functional_data

scan_idx scan ID,segmentation segmentation id,postsyn segment id unique within each Segmentation,von_pref preferred directions
3,1,73396353,5.51174
3,1,10914339,3.22457
3,1,58281524,4.01851
3,1,77127834,4.7786
3,1,29789967,2.29286
3,1,59033263,0.15092
3,1,78665518,2.90859
3,1,37086820,2.477
3,1,61375585,2.36653
3,1,81578447,3.34475


In [23]:
#gets just the names of the unique significantly tuned post-synaptic segments
postsyns = (dj.U('postsyn') & functional_data)
postsyns

postsyn segment id unique within each Segmentation
73396353
10914339
58281524
77127834
29789967
59033263
78665518
37086820
61375585
81578447


In [27]:
#restrict my initial table only by the targets that have functionally tuned data
unique_axons_tuning_only = axons_from_unique_axon_trace_pairs_table & postsyns
unique_axons_tuning_only

presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,n_syn calculated attribute
9117,82251040,1
18695,82251040,1
21736,82251040,1
31276,82251040,1
58208,10914339,1
71995,39500491,1
71995,81578447,1
71995,82251040,1
99938,82251040,1
117428,82251040,1


In [25]:
#unique axons without the restriction
unique_axons = dj.U('presyn') & axons_from_unique_axon_trace_pairs_table
unique_axons

presyn segment id unique within each Segmentation
108
9117
18695
20560
21736
27408
31276
35065
35324
53254


In [28]:
#unique axons for the functionallly tuned axons
unique_axons_funct_tuned = dj.U('presyn') & unique_axons_tuning_only
unique_axons_funct_tuned

#a reduction from 53,000 to 8,000
unique_axons_funct_tuned = dj.U('postsyn') & unique_axons_tuning_only
unique_axons_funct_tuned

postsyn segment id unique within each Segmentation
82251040
10914339
39500491
81578447
78665518
107860456
48234238
67981253
942272
